In [1]:
from openbb import obb

obb.equity.price.historical("AAPL", start_date="2020-01-01", end_date="2020-01-31")

Extensions to add: yfinance@1.4.4
Extensions to remove: yfinance@1.4.3

Building...


ImportError: cannot import name 'EquityInfo' from 'typing' (/Users/jjfantini/.local/share/uv/python/cpython-3.12.10-macos-aarch64-none/lib/python3.12/typing.py)